In [1]:
!pip install langchain faiss-cpu sentence-transformers transformers


Defaulting to user installation because normal site-packages is not writeable


DEPRECATION: Loading egg at c:\programdata\anaconda3\lib\site-packages\vboxapi-1.0-py3.11.egg is deprecated. pip 25.1 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330

[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.text_splitter import CharacterTextSplitter
from langchain.llms import HuggingFacePipeline
from langchain.chains import RetrievalQA
from langchain.docstore.document import Document
from transformers import pipeline

import os

In [3]:
def load_and_chunk(file_path, chunk_size=500, chunk_overlap=50):
    with open(file_path,'r',encoding='utf-8') as f:
        text=f.read()
    
    splitter=CharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
    chunks=splitter.split_text(text)
    
    return [Document(page_content=chunk) for chunk in chunks]
docs=load_and_chunk("C:\\Users\\Akshata Pandit\\Downloads\\knowledge_base.txt")
print(f"Loaded {len(docs)} chunks.")

Loaded 20 chunks.


In [4]:
def create_vector_store(docs):
    embeddings=HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
    vectorstore=FAISS.from_documents(docs, embeddings)
    return vectorstore

vectorstore=create_vector_store(docs)

C:\Users\Akshata Pandit\AppData\Local\Temp\ipykernel_11888\318331682.py:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings=HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")


In [5]:
pipe=pipeline("text2text-generation",model="google/flan-t5-base",max_length=256)
llm=HuggingFacePipeline(pipeline=pipe)
rag_chain=RetrievalQA.from_chain_type(
    llm=llm,
    retriever=vectorstore.as_retriever(),
    return_source_documents=True
)

Device set to use cpu
C:\Users\Akshata Pandit\AppData\Local\Temp\ipykernel_11888\375446207.py:2: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  llm=HuggingFacePipeline(pipeline=pipe)


In [6]:
def ask_question(query):
    result=rag_chain({"query": query})
    answer=result["result"]
    sources=result["source_documents"]
    
    print("Answer:")
    print(answer)
    print("\nSource Chunks:")
    for i, doc in enumerate(sources):
        print(f"\n--- Source {i+1} ---\n{doc.page_content}")
        
    return answer
ask_question("What is retrieval augmented generation?")

C:\Users\Akshata Pandit\AppData\Local\Temp\ipykernel_11888\799565735.py:2: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result=rag_chain({"query": query})


Answer:
an architecture that enhances language model output by retrieving relevant documents from a knowledge base before answering a query

Source Chunks:

--- Source 1 ---
Retrieval-Augmented Generation (RAG) is an architecture that enhances language model output by retrieving relevant documents from a knowledge base before answering a query. It combines a retriever and a generator model.

RAG systems are particularly useful for tasks requiring up-to-date or domain-specific information, such as customer support, document Q&A, legal search, and medical diagnostics.

--- Source 2 ---
Using metadata filters (like document type or section) during retrieval can improve relevance and precision, especially in multi-domain knowledge bases.

Hybrid retrieval combines dense embeddings with keyword-based search (BM25). This is useful when embeddings alone are insufficient for precision.

Multi-vector retrieval stores multiple embeddings per document, such as one per sentence or paragraph. This 

'an architecture that enhances language model output by retrieving relevant documents from a knowledge base before answering a query'

In [7]:
import gradio as gr

def chatbot(query):
    result=rag_chain({"query": query})
    answer=result["result"]
    sources="\n\n".join([f"Source {i+1}: {doc.page_content}" for i, doc in enumerate(result["source_documents"])])
    return f"{answer}\n\n--- Sources ---\n{sources}"

gr.Interface(fn=chatbot,
             inputs=gr.Textbox(lines=2, placeholder="Ask a question..."),
             outputs="text",
             title="RAG Chatbot",
             description="Retrieval-Augmented Generation using FAISS + FLAN-T5"
).launch()


* Running on local URL:  http://127.0.0.1:7864
* To create a public link, set `share=True` in `launch()`.


In [8]:
def chatbot(query):
    result=rag_chain({"query": query})
    answer=result["result"]
    sources="\n\n".join([f"Source {i+1}: {doc.page_content}" for i, doc in enumerate(result["source_documents"])])
    return f"{answer}\n\n--- Sources ---\n{sources}"

gr.Interface(fn=chatbot,
             inputs=gr.Textbox(lines=2, placeholder="Ask a question..."),
             outputs="text",
             title="RAG Chatbot",
             description="Retrieval-Augmented Generation using FAISS + FLAN-T5"
).launch()

* Running on local URL:  http://127.0.0.1:7865
* To create a public link, set `share=True` in `launch()`.


In [10]:
def chatbot(query):
    result=rag_chain({"query": query})
    answer=result["result"]
    sources="\n\n".join([f"Source {i+1}: {doc.page_content}" for i, doc in enumerate(result["source_documents"])])
    return f"{answer}\n\n--- Sources ---\n{sources}"

gr.Interface(fn=chatbot,
             inputs=gr.Textbox(lines=2, placeholder="Ask a question..."),
             outputs="text",
             title="RAG Chatbot",
             description="Retrieval-Augmented Generation using FAISS + FLAN-T5"
).launch()

* Running on local URL:  http://127.0.0.1:7867
* To create a public link, set `share=True` in `launch()`.
